In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
from IPython.display import Markdown,display

In [2]:
load_dotenv(override = True)

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print("OpenAI key loaded!!")

if google_api_key:
    print("Gemini key loaded!!")

OpenAI key loaded!!
Gemini key loaded!!


In [4]:
openai = OpenAI()
MODEL="gpt-4o-mini"

In [5]:
system_prompt = """
You are a funny and helpful assistant that answers questions in a clever and detailed way.
If you do not know the answer of any question, please say so. Do not provide wrong answers.
Respond in Markdown
"""                 

In [6]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response  

In [7]:
print(gr.themes)

<module 'gradio.themes' from 'c:\\Users\\KIIT\\anaconda3\\envs\\llms\\Lib\\site-packages\\gradio\\themes\\__init__.py'>


In [8]:
gr.__version__

'5.33.0'

In [9]:
chatbot = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Auralis ",
    description="Ask me anything! I'm here to help with a touch of humor.",
    theme="soft",
    chatbot=gr.Chatbot(
        avatar_images=("user_avatar.png", "bot_avatar.png"),
        show_label=True,
        type="messages"
    )
).launch(share=True, pwa=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://49c8fa3d1735587e3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response}
        ]

with gr.Blocks(theme="soft") as demo:
    gr.Markdown(
        """
        <div style='display: flex; flex-direction: column; align-items: center; margin-top: 100px;'>
            <h1 style='font-size: 2.5em; font-weight: 600;'>Auralis</h1>
            <h2 style='font-size: 2.0em; font-weight: 500;'>What are you working on?</h2>
        </div>
        """,
        elem_id="main-title"
    )
    chatbot = gr.Chatbot(
        avatar_images=("user_avatar.png", "bot_avatar.png"),
        show_label=False,
        height=400,
        type="messages"
    )
    with gr.Row():
        msg = gr.Textbox(
            show_label=False,
            placeholder="Ask anything",
            scale=8,
            elem_id="input-box",
            autofocus=True,
            container=False,
            type="text"
        )
        send_icon = gr.Button(value="\U0001F4E7", scale=1, elem_id="send-btn")
    msg.submit(chat, [msg, chatbot], chatbot, queue=True, scroll_to_output=True, show_progress=True, api_name=None)
    send_icon.click(chat, [msg, chatbot], chatbot, queue=True, scroll_to_output=True, show_progress=True, api_name=None)

demo.launch(share=True, pwa=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://5e7c89d9166c3ebe2e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Check why the diffence in:
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response}
        ]

In [11]:
chatbot = gr.ChatInterface(fn=chat, type="messages").launch(share=True,pwa=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://731b962f2e7d621421.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Image Generation using DALL-E-3


In [12]:
import base64
from io import BytesIO
from PIL import Image

In [13]:
def artist(city):
    image_reponse = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}.",
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_reponse.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [14]:
# image = artist("Kolkata")
# display(image)

In [15]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

c:\Users\KIIT\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1028: UserWarning: Expected 2 arguments for function <function chat at 0x0000028BD9FDC860>, received 1.
  warnings.warn(
c:\Users\KIIT\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1032: UserWarning: Expected at least 2 arguments for function <function chat at 0x0000028BD9FDC860>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [16]:
keywords = ['create', 'visual', 'image', 'generate', 'picture']

In [17]:
system_prompt = "You are Auralis, a helpful and creative assistant."

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    if any(keyword in message.lower() for keyword in keywords):
        if not message or len(message.strip()) < 5:
            return history + [{"role": "user", "content": message}, {"role": "assistant", "content": "Please provide a more detailed prompt to generate an image."}], None

        image_response = openai.images.generate(
            model="dall-e-3",
            prompt=message,
            size="1024x1024",
            n=1,
            response_format="b64_json"
        )

        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        image = Image.open(BytesIO(image_data))

        return history + [{"role": "user", "content": message}, {"role": "assistant", "content": "Here is your image!"}], image

    # Handle regular chat stream
    response = ""
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response}
        ], None

# === GRADIO UI ===
with gr.Blocks(theme="soft") as demo:
    gr.Markdown(
        """
        <div style='display: flex; flex-direction: column; align-items: center; margin-top: 100px;'>
            <h1 style='font-size: 2.5em; font-weight: 600;'>Auralis</h1>
            <h2 style='font-size: 2.0em; font-weight: 500;'>What are you working on?</h2>
        </div>
        """,
        elem_id="main-title"
    )

    chatbot = gr.Chatbot(
        avatar_images=("user_avatar.png", "bot_avatar.png"),
        show_label=False,
        height=400,
        type="messages"
    )
    image_output = gr.Image(visible=False)

    with gr.Row():
        msg = gr.Textbox(
            show_label=False,
            placeholder="Ask anything",
            scale=8,
            elem_id="input-box",
            autofocus=True,
            container=False,
            type="text"
        )
        send_icon = gr.Button(value="\U0001F4E7", scale=1, elem_id="send-btn")

    def process_input(message, history):
        result = chat(message, history)
        if isinstance(result, tuple):
            return result  # history, image (for image prompts)
        else:
            for updated_history, _ in result:
                yield updated_history, None

    msg.submit(process_input, [msg, chatbot], [chatbot, image_output], queue=True, scroll_to_output=True, show_progress=True)
    send_icon.click(process_input, [msg, chatbot], [chatbot, image_output], queue=True, scroll_to_output=True, show_progress=True)

demo.launch(share=True, pwa=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://449599858bb5a8ba30.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [31]:
import gradio as gr
import openai
import base64
from PIL import Image
from io import BytesIO
import os

# Configuration
MODEL = "gpt-4o-mini"  # or your preferred model
openai.api_key = os.getenv("OPENAI_API_KEY")  # Make sure to set your API key

# System prompt for the chatbot
system_prompt = """
You are Auralis, a helpful and a funny AI assistant who gives detailed responses. You can engage in conversations on a 
wide range of topics and help users with various tasks. Be friendly and informative in your responses.
"""

# Keywords that trigger image generation
keywords = ['create', 'visual', 'image', 'generate', 'picture', 'draw', 'make', 'design', 'art', 'illustration']

def generate_image(prompt):
    """Generate image using DALL-E and return PIL Image object"""
    try:
        # Clean and validate the prompt
        clean_prompt = prompt.strip()
        if len(clean_prompt) > 1000:  # DALL-E has prompt limits
            clean_prompt = clean_prompt[:1000]
        
        # Add some delay to avoid rate limiting
        import time
        time.sleep(1)
        
        print(f"Generating image with prompt: {clean_prompt}")  # Debug logging
        
        image_response = openai.images.generate(
            model="dall-e-3",
            prompt=clean_prompt,
            size="1024x1024",
            n=1,
            response_format="b64_json",
            quality="standard"  # Use standard quality to reduce costs and server load
        )
        
        # Process the image
        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        
        # Convert to PIL Image
        image = Image.open(BytesIO(image_data))
        print("Image generated successfully!")  # Debug logging
        return image, None
        
    except openai.RateLimitError as e:
        error_msg = "Rate limit exceeded. Please wait a moment before generating another image."
        print(f"Rate limit error: {e}")
        return None, error_msg
    except openai.APIError as e:
        error_msg = f"OpenAI API error: {e.message if hasattr(e, 'message') else str(e)}"
        print(f"API error: {e}")
        return None, error_msg
    except Exception as e:
        error_msg = f"Unexpected error generating image: {str(e)}"
        print(f"Unexpected error: {e}")
        return None, error_msg

def generate_text_response_streaming(message, history):
    """Generate streaming text response using OpenAI API"""
    try:
        messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
        
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=True
        )
        
        # Yield chunks of the response as they come in
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                yield chunk.choices[0].delta.content
                
    except Exception as e:
        yield f"Sorry, I encountered an error: {str(e)}"

def chat_interface_streaming(message, history):
    """Main chat interface function with streaming support"""
    if not message or not message.strip():
        yield history, None
        return
    
    # Check if we should generate an image
    should_generate_image = any(keyword in message.lower() for keyword in keywords)
    
    if should_generate_image:
        # More robust validation for image generation
        clean_message = message.strip()
        if len(clean_message) < 10:
            error_msg = "Please provide a more detailed description (at least 10 characters) to generate a high-quality image."
            new_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": error_msg}]
            yield new_history, None
            return
        
        # Check for potentially problematic content
        forbidden_words = ['nsfw', 'nude', 'explicit', 'violence', 'blood', 'gore']
        if any(word in clean_message.lower() for word in forbidden_words):
            error_msg = "I can't generate images with that content. Please try a different, more appropriate prompt."
            new_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": error_msg}]
            yield new_history, None
            return
        
        # Show generating message
        generating_msg = f"🎨 Generating image for: '{clean_message}'\nThis may take a few moments..."
        temp_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": generating_msg}]
        yield temp_history, None
        
        # Generate image
        image, error = generate_image(clean_message)
        
        if error:
            # Return error message with suggestions
            if "rate limit" in error.lower():
                error_msg = f"{error}\n\n💡 Tip: Try waiting 30-60 seconds before generating another image."
            elif "server" in error.lower():
                error_msg = f"{error}\n\n💡 Tip: OpenAI servers might be busy. Try again in a few minutes, or try a simpler prompt."
            else:
                error_msg = f"{error}\n\n💡 Tip: Try rephrasing your prompt or making it more specific."
            
            new_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": error_msg}]
            yield new_history, None
        else:
            # Return success message with image
            response_text = f"✨ Here's your generated image: '{clean_message}'"
            new_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": response_text}]
            yield new_history, image
    
    else:
        # Generate streaming text response
        new_history = history + [{"role": "user", "content": message}, {"role": "assistant", "content": ""}]
        
        assistant_message = ""
        for text_chunk in generate_text_response_streaming(message, history):
            assistant_message += text_chunk
            # Update the last message in history with the accumulated response
            new_history[-1]["content"] = assistant_message
            yield new_history, None

def submit_message(message, history):
    """Handle message submission with streaming support"""
    if not message.strip():
        return history, None, ""
    
    # Use the streaming chat interface
    for new_history, image in chat_interface_streaming(message, history):
        yield new_history, image, ""  # Clear input field

def clear_chat():
    """Clear the chat history"""
    return [], None

# Custom CSS for better styling
css = """
#main-title {
    text-align: center;
    margin-bottom: 30px;
}

#input-box {
    border-radius: 25px !important;
    border: 2px solid #e0e0e0 !important;
    padding: 12px 20px !important;
    font-size: 16px !important;
}

#send-btn {
    border-radius: 50% !important;
    width: 50px !important;
    height: 50px !important;
    min-width: 50px !important;
    background-color: white !important;
    border: 2px solid #e0e0e0 !important;
    color: #333 !important;
    font-size: 18px !important;
    margin-left: 10px !important;
    display: flex !important;
    align-items: center !important;
    justify-content: center !important;
}

#send-btn:hover {
    transform: scale(1.05) !important;
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1) !important;
    border-color: #ccc !important;
}

.chatbot {
    border-radius: 15px !important;
    border: 1px solid #e0e0e0 !important;
}

#clear-btn {
    background-color: #ff6b6b !important;
    color: white !important;
    border: none !important;
    border-radius: 20px !important;
    padding: 8px 16px !important;
    margin-bottom: 10px !important;
}

#clear-btn:hover {
    background-color: #ff5252 !important;
    transform: scale(1.02) !important;
}

.image-container {
    border-radius: 15px !important;
    border: 1px solid #e0e0e0 !important;
    margin-top: 10px !important;
}
"""

# Create the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), css=css, title="Auralis - AI Assistant") as demo:
    # Header
    gr.Markdown(
        """
        <div style='display: flex; flex-direction: column; align-items: center; margin-top: 100px;'>
            <h1 style='font-size: 4.0em; font-weight: 600;'>Auralis</h1>
            <h2 style='font-size: 2.0em; font-weight: 500;'>What are you working on?</h2>
        </div>
        """,
        elem_id="main-title"
    )
    
    # Clear button
    with gr.Row():
        clear_button = gr.Button("🗑️ Clear Chat", elem_id="clear-btn", size="sm")
    
    # Chat interface
    chatbot = gr.Chatbot(
        show_label=False,
        height=500,
        type="messages",
        avatar_images=("user_avatar.png", "bot_avatar.png"),
        elem_classes=["chatbot"]
    )
    
    # Image output (will show generated images)
    image_output = gr.Image(
        label="Generated Image", 
        visible=True, 
        elem_classes=["image-container"],
        height=400  # Increased height for better visibility
    )
    
    # Input section
    with gr.Row():
        msg = gr.Textbox(
            show_label=False,
            placeholder="Type your message here... (use words like 'create', 'generate', 'draw' for images)",
            scale=9,
            elem_id="input-box",
            autofocus=True,
            container=False,
            lines=1,
            max_lines=3
        )
        send_btn = gr.Button("↑", scale=1, elem_id="send-btn")
    
    # Event handlers
    msg.submit(
        submit_message,
        inputs=[msg, chatbot],
        outputs=[chatbot, image_output, msg],
        queue=True
    )
    
    send_btn.click(
        submit_message,
        inputs=[msg, chatbot],
        outputs=[chatbot, image_output, msg],
        queue=True
    )
    
    # Clear chat functionality
    clear_button.click(
        clear_chat,
        outputs=[chatbot, image_output],
        queue=False
    )
    
    # Example prompts
    gr.Examples(
        examples=[
            ["What is artificial intelligence?"],
            ["Create a beautiful sunset over mountains"],
            ["Explain quantum computing in simple terms"],
            ["Generate an image of a futuristic city"],
            ["What are the benefits of renewable energy?"],
            ["Draw a cute robot helping humans"]
        ],
        inputs=msg,
        label="Try these examples:"
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True, pwa=True)

* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://871df99184abdc02b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating image with prompt: Create a beautiful sunset over mountains
API error: Error code: 500 - {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}}


### Audio Generation

In [20]:
from pydub import AudioSegment
from pydub.playback import play


In [21]:

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [22]:
talker("Well, hi there")


c:\Users\KIIT\anaconda3\envs\llms\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

### Working with Gemini

In [12]:
import google.generativeai


In [13]:
user_prompt="""
Give me a detailed analysis of what is google is what it does!!
"""

In [14]:
from IPython.display import display, Markdown, clear_output
import time

In [15]:
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_prompt
)
response = gemini.generate_content(user_prompt, stream=True)

content = ''
for chunk in response:
    content += chunk.text
    clear_output(wait=True)
    display(Markdown(content))
    time.sleep(0.05)

Alright, buckle up, buttercup, because we're about to dive deep into the Google-verse!

**What is Google? A Multi-Faceted Entity**

On the surface, Google is a search engine. But that's like saying the Mona Lisa is just a painting. Google has evolved into a sprawling, tentacled, digital octopus with its suckers in pretty much everything. Here's a breakdown:

*   **The OG: The Search Engine:** This is the granddaddy of them all. You type in a query, and Google's algorithms (a super-secret sauce of mathematical wizardry) crawl the entire internet to give you a ranked list of relevant websites. It's how most people start their online journey.

*   **A Software Juggernaut:** Google's not just about search. They have a whole suite of software products:

    *   **Android:** The most popular mobile operating system in the world. If you have a non-Apple smartphone, chances are it's running Android.
    *   **Chrome:** A web browser that's practically synonymous with "internet."
    *   **Gmail:** Email service with a huge storage capacity.
    *   **Google Docs, Sheets, Slides:** Their answer to Microsoft Office, but free and cloud-based.
    *   **Google Maps:** A mapping service which most people rely on to navigate these days.
    *   **YouTube:** The world's biggest video-sharing platform.
    *   **Google Drive:** Cloud storage, so you can keep your files safe and access them from anywhere.
    *   **Google Photos:** Photo storage and organization.
    *   **Google Meet:** Video conferencing, similar to Zoom.
    *   **Google Calendar:** Scheduling and reminders.
    *   **Google Translate:** The translator.
    *   **Google News:** A news aggregator.

*   **Hardware Player:** Google makes stuff you can touch.

    *   **Pixel Phones:** Their own line of smartphones.
    *   **Nest Products:** Smart home devices like thermostats, security cameras, and doorbells.
    *   **Chromecast:** A device that lets you stream content from your phone or computer to your TV.
    *   **Google Wifi:** Mesh wifi systems for better home internet coverage.

*   **AI and Machine Learning Pioneer:** Google is heavily invested in artificial intelligence and machine learning:

    *   **Google AI:** A research division dedicated to pushing the boundaries of AI.
    *   **TensorFlow:** An open-source machine learning framework used by researchers and developers worldwide.
    *   **Google Assistant:** A virtual assistant that can answer questions, set alarms, control smart home devices, and more.
    *   **Bard:** Google's answer to Chat GPT.

*   **A Data Colossus:** Google collects a *lot* of data. Every search you make, every website you visit (if they have Google Analytics installed), every video you watch on YouTube – it's all data that Google can use to personalize ads, improve its services, and build better AI models. This is both incredibly useful and a little bit creepy, depending on your perspective.

**What Does Google Do? The Core Functions**

So, what does Google *do* with all this power and reach?

*   **Organizes Information:** Google's primary mission is to "organize the world's information and make it universally accessible and useful." The search engine is the flagship manifestation of this mission, but it extends to Google Scholar (for academic research), Google Books (for digitizing books), and other projects.

*   **Connects People to Information:** It's not just about organizing data; it's about getting the right information to the right people at the right time. Google's algorithms are constantly being refined to deliver more relevant and personalized search results.

*   **Monetizes Through Advertising:** The vast majority of Google's revenue comes from advertising. They sell ad space on their search results pages, on YouTube, on websites that use Google AdSense, and within their mobile apps. The more targeted the ads, the more valuable they are.

*   **Innovates:** Google is constantly experimenting with new technologies and products. Some of these efforts become wildly successful (like Android), while others fade away (remember Google Glass?). They're willing to take risks and invest in the future.

*   **Shapes the Internet:** Google has a massive influence on how the internet works. Their search algorithms can make or break websites. Their open-source projects like TensorFlow and Kubernetes shape the development landscape. And their decisions about privacy and security have a ripple effect across the entire web.

**In a Nutshell**

Google is more than just a search engine; it's a technological powerhouse that touches nearly every aspect of our digital lives. It's a company that organizes information, connects people, monetizes data, innovates relentlessly, and shapes the internet as we know it.

**Disclaimer:** This is a simplified explanation, of course. The inner workings of Google are complex and constantly evolving. But hopefully, this gives you a comprehensive overview of what Google is and what it does.
